<a href="https://colab.research.google.com/github/kridtapon/WFO-SureSignal-MACD/blob/main/WFO_SureSignal_MACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate KST (Know Sure Thing)
def calculate_kst(data, r1=10, r2=15, r3=20, r4=30, sm1=10, sm2=10, sm3=10, sm4=15, signal_period=9):
    roc1 = data['Close'].pct_change(r1) * 100
    roc2 = data['Close'].pct_change(r2) * 100
    roc3 = data['Close'].pct_change(r3) * 100
    roc4 = data['Close'].pct_change(r4) * 100

    kst = (roc1.rolling(sm1).mean() +
           roc2.rolling(sm2).mean() * 2 +
           roc3.rolling(sm3).mean() * 3 +
           roc4.rolling(sm4).mean() * 4)

    signal = kst.rolling(signal_period).mean()
    return kst, signal

# Function to calculate MACD
def calculate_macd(data, fast=12, slow=26, signal_period=9):
    ema_fast = data['Close'].ewm(span=fast, min_periods=1).mean()
    ema_slow = data['Close'].ewm(span=slow, min_periods=1).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_period, min_periods=1).mean()
    return macd, signal

# Walk-forward optimization with KST and MACD
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Parameter grids for MACD and KST
    macd_fast_grid = range(5, 15, 2)
    macd_slow_grid = range(15, 31, 2)
    macd_signal_grid = range(5, 31, 2)
    kst_signal_grid = range(5, 31, 2)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through parameter combinations
        for params in itertools.product(macd_fast_grid, macd_slow_grid, macd_signal_grid, kst_signal_grid):
            macd_fast, macd_slow, macd_signal, kst_signal = params

            # Calculate indicators on the training data
            train_data['MACD'], train_data['MACD_Signal'] = calculate_macd(train_data, fast=macd_fast, slow=macd_slow, signal_period=macd_signal)
            train_data['KST'], train_data['KST_Signal'] = calculate_kst(train_data, signal_period=kst_signal)

            # Generate entry and exit signals
            entries = (train_data['MACD'] > train_data['MACD_Signal']) & (train_data['KST'] > train_data['KST_Signal'])
            exits = (train_data['MACD'] < train_data['MACD_Signal']) & (train_data['KST'] < train_data['KST_Signal'])

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = params

        # Test with the best parameters on the test data
        test_data['MACD'], test_data['MACD_Signal'] = calculate_macd(test_data, fast=best_params[0], slow=best_params[1], signal_period=best_params[2])
        test_data['KST'], test_data['KST_Signal'] = calculate_kst(test_data, signal_period=best_params[3])

        entries = (test_data['MACD'] > test_data['MACD_Signal']) & (test_data['KST'] > test_data['KST_Signal'])
        exits = (test_data['MACD'] < test_data['MACD_Signal']) & (test_data['KST'] < test_data['KST_Signal'])

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'GDDY'
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combined entry/exit signals
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    yearly_data['MACD'], yearly_data['MACD_Signal'] = calculate_macd(yearly_data, fast=params[0], slow=params[1], signal_period=params[2])
    yearly_data['KST'], yearly_data['KST_Signal'] = calculate_kst(yearly_data, signal_period=params[3])

    entries = (yearly_data['MACD'] > yearly_data['MACD_Signal']) & (yearly_data['KST'] > yearly_data['KST_Signal'])
    exits = (yearly_data['MACD'] < yearly_data['MACD_Signal']) & (yearly_data['KST'] < yearly_data['KST_Signal'])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-3-2f46d03210eb>:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-2f46d03210eb>:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-2f46d03210eb>:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in


Walk-Forward Optimization Results:
   Year      Best_Params  Performance
0  2020  (13, 15, 27, 5)     0.465961
1  2021  (5, 27, 29, 11)     0.205681
2  2022  (11, 19, 23, 5)    -0.154165
3  2023   (9, 15, 15, 5)     0.167678
4  2024   (9, 15, 15, 5)     0.487194
5  2025   (9, 15, 11, 7)     0.000000
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           239337.396811
Total Return [%]                       139.337397
Benchmark Return [%]                   187.879233
Max Gross Exposure [%]                      100.0
Total Fees Paid                       8524.975881
Max Drawdown [%]                        29.304098
Max Drawdown Duration                       603.0
Total Trades                                   28
Total Closed Trades                            28
Total Open Trades                               

<ipython-input-3-2f46d03210eb>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-2f46d03210eb>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-2f46d03210eb>:127: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-2f46d